In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
import requests
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from itertools import combinations
from collections import OrderedDict 

%matplotlib inline

/Users/scottlew/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
years = [x for x in range(1876,2019,1)]
years

[1876,
 1877,
 1878,
 1879,
 1880,
 1881,
 1882,
 1883,
 1884,
 1885,
 1886,
 1887,
 1888,
 1889,
 1890,
 1891,
 1892,
 1893,
 1894,
 1895,
 1896,
 1897,
 1898,
 1899,
 1900,
 1901,
 1902,
 1903,
 1904,
 1905,
 1906,
 1907,
 1908,
 1909,
 1910,
 1911,
 1912,
 1913,
 1914,
 1915,
 1916,
 1917,
 1918,
 1919,
 1920,
 1921,
 1922,
 1923,
 1924,
 1925,
 1926,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1939,
 1940,
 1941,
 1942,
 1943,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018]

## Note data was scraped before, data will be concatenated 

## Team pitching & batting stats from 1876-2018 taken from baseball reference.com

In [4]:
#years = [2011,2012,2013,2014,2015,2016,2017,2018]
file_name = "MLB_Pitch_{}.csv"
dfs_p = []
for year in years:
    file_name = "MLB_Pitch_{}.csv"
    file_path = file_name.format(year)
    dfs_p.append(pd.read_csv(file_path))
df_pitch_1876_2018 = pd.concat(dfs_p)

In [5]:
df_pitch_1876_2018.head(3)

,#P,BB,BB9,BF,BK,CG,ER,ERA,ERA+,FIP,...,SO9,SV,Tm,W,W-L%,WHIP,WP,Year,cSho,tSho
0,5,41,0.7,2565,0,53,197,3.22,73,2.42,...,0.4,2,ATH,14,0.237,1.498,33,1876,1.0,1.0
1,6,104,1.5,2834,0,49,176,2.51,89,2.63,...,1.1,7,BSN,39,0.557,1.323,46,1876,3.0,3.0
2,5,29,0.4,2489,0,58,116,1.76,137,2.34,...,0.8,4,CHC,52,0.788,1.075,3,1876,8.0,9.0


In [6]:
df_pitch_1876_2018.tail(3)

,#P,BB,BB9,BF,BK,CG,ER,ERA,ERA+,FIP,...,SO9,SV,Tm,W,W-L%,WHIP,WP,Year,cSho,tSho
27,32,491,3.1,6231,8,1,783,4.92,99,4.79,...,7.1,42,TEX,67,0.414,1.403,57,2018,0.0,5.0
28,36,551,3.5,6265,5,0,772,4.85,87,4.53,...,8.1,39,TOR,73,0.451,1.414,71,2018,0.0,3.0
29,31,487,3.0,6087,6,2,649,4.04,106,4.15,...,8.8,40,WSN,82,0.506,1.250,42,2018,1.0,7.0


In [7]:
df_pitch_1876_2018.shape

(2845, 37)

In [8]:
df_pitch_1876_2018.dtypes

#P        int64
BB        int64
BB9     float64
BF        int64
BK        int64
CG        int64
ER        int64
ERA     float64
ERA+      int64
FIP     float64
G         int64
GF        int64
GS        int64
H         int64
H9      float64
HBP     float64
HR        int64
HR9     float64
IBB     float64
IP      float64
L         int64
LOB     float64
PAge    float64
R         int64
RA/G    float64
SO        int64
SO/W    float64
SO9     float64
SV        int64
Tm       object
W         int64
W-L%    float64
WHIP    float64
WP        int64
Year      int64
cSho    float64
tSho    float64
dtype: object

# Batting stats from seasons 1876-2018

In [9]:
#years = [2011,2012,2013,2014,2015,2016,2017,2018]
file_name = "MLB_BAT_{}.csv"
dfs_b = []
for year in years:
    file_name = "MLB_BAT_{}.csv"
    file_path = file_name.format(year)
    dfs_b.append(pd.read_csv(file_path))
df_bat_1876_2018 = pd.concat(dfs_b)

In [10]:
df_bat_1876_2018.head(3)

,Tm,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,...,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB,Year
0,ATH,21,26.0,6.30,60,2414,2387,378,646,79,...,0.621,104,816,NaN,NaN,NaN,NaN,NaN,NaN,1876
1,BSN,17,24.7,6.73,70,2780,2722,471,723,96,...,0.609,103,894,NaN,NaN,NaN,NaN,NaN,NaN,1876
2,CHC,11,25.9,9.45,66,2818,2748,624,926,131,...,0.770,146,1145,NaN,NaN,NaN,NaN,NaN,NaN,1876


In [11]:
df_bat_1876_2018.tail(3)

,Tm,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,...,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB,Year
27,TEX,50,27.4,4.55,162,6163,5453,737,1308,266,...,0.722,88,2204,104.0,88.0,33.0,34.0,16.0,1093.0,2018
28,TOR,63,28.9,4.38,162,6076,5477,709,1336,320,...,0.739,103,2339,118.0,58.0,5.0,37.0,15.0,1036.0,2018
29,WSN,53,27.6,4.76,162,6288,5517,771,1402,284,...,0.753,98,2309,104.0,59.0,41.0,40.0,56.0,1180.0,2018


In [12]:
df_bat_1876_2018.shape

(2845, 30)

In [13]:
df_bat_1876_2018.dtypes

Tm         object
#Bat        int64
BatAge    float64
R/G       float64
G           int64
PA          int64
AB          int64
R           int64
H           int64
2B          int64
3B          int64
HR          int64
RBI       float64
SB        float64
CS        float64
BB          int64
SO          int64
BA        float64
OBP       float64
SLG       float64
OPS       float64
OPS+        int64
TB          int64
GDP       float64
HBP       float64
SH        float64
SF        float64
IBB       float64
LOB       float64
Year        int64
dtype: object

## Join dataframes to each other using JOIN ON 'Tm' & 'Year'

In [14]:
#df = pd.merge(dfb, dfp, on= ['Tm','Year'])
df_1876_2018 = pd.merge(df_bat_1876_2018,df_pitch_1876_2018, on=['Tm','Year'])

In [15]:
df_1876_2018.head(12)

,Tm,#Bat,BatAge,R/G,G_x,PA,AB,R_x,H_x,2B,...,SO_y,SO/W,SO9,SV,W,W-L%,WHIP,WP,cSho,tSho
0,ATH,21,26.0,6.30,60,2414,2387,378,646,79,...,22,0.54,0.4,2,14,0.237,1.498,33,1.0,1.0
1,BSN,17,24.7,6.73,70,2780,2722,471,723,96,...,77,0.74,1.1,7,39,0.557,1.323,46,3.0,3.0
2,CHC,11,25.9,9.45,66,2818,2748,624,926,131,...,51,1.76,0.8,4,52,0.788,1.075,3,8.0,9.0
3,CIN,13,24.5,3.66,65,2413,2372,238,555,51,...,60,1.76,0.9,0,9,0.138,1.496,28,0.0,0.0
4,HAR,12,26.3,6.22,69,2703,2664,429,711,96,...,114,4.22,1.6,0,47,0.691,0.957,15,11.0,11.0
5,LOU,16,24.4,4.06,69,2594,2570,280,641,68,...,125,3.29,1.7,0,30,0.455,1.000,25,5.0,5.0
6,NYU,24,24.9,4.56,57,2198,2180,260,494,39,...,37,1.54,0.6,0,21,0.375,1.400,3,2.0,2.0
7,STL,10,26.0,6.03,64,2537,2478,386,642,73,...,103,2.64,1.6,0,45,0.703,0.886,34,16.0,16.0
8,BSN,12,26.1,6.87,61,2433,2368,419,700,91,...,177,4.66,2.9,0,42,0.700,1.086,20,7.0,7.0
9,CHC,18,25.5,6.10,60,2330,2273,366,633,79,...,92,1.59,1.6,3,26,0.441,1.288,45,3.0,3.0


In [16]:
df_1876_2018.tail(3)

,Tm,#Bat,BatAge,R/G,G_x,PA,AB,R_x,H_x,2B,...,SO_y,SO/W,SO9,SV,W,W-L%,WHIP,WP,cSho,tSho
2812,TEX,50,27.4,4.55,162,6163,5453,737,1308,266,...,1121,2.28,7.1,42,67,0.414,1.403,57,0.0,5.0
2813,TOR,63,28.9,4.38,162,6076,5477,709,1336,320,...,1298,2.36,8.1,39,73,0.451,1.414,71,0.0,3.0
2814,WSN,53,27.6,4.76,162,6288,5517,771,1402,284,...,1417,2.91,8.8,40,82,0.506,1.250,42,1.0,7.0


In [17]:
df_1876_2018.shape

(2815, 65)

## Pitching & Batting Stats for 1876-2018 combined and merged

## Save the stats for 1876-2018 as a csv file for future use.

In [19]:
# Save result df as as csv file for future use!
df_1876_2018.to_csv('MLB_STATS_1876-2018_BP.csv', index = False)

In [20]:
df = pd.read_csv('MLB_STATS_1876-2018_BP.csv')

In [21]:
df.head(3)

,Tm,#Bat,BatAge,R/G,G_x,PA,AB,R_x,H_x,2B,...,SO_y,SO/W,SO9,SV,W,W-L%,WHIP,WP,cSho,tSho
0,ATH,21,26.0,6.30,60,2414,2387,378,646,79,...,22,0.54,0.4,2,14,0.237,1.498,33,1.0,1.0
1,BSN,17,24.7,6.73,70,2780,2722,471,723,96,...,77,0.74,1.1,7,39,0.557,1.323,46,3.0,3.0
2,CHC,11,25.9,9.45,66,2818,2748,624,926,131,...,51,1.76,0.8,4,52,0.788,1.075,3,8.0,9.0


In [22]:
df.tail(3)

,Tm,#Bat,BatAge,R/G,G_x,PA,AB,R_x,H_x,2B,...,SO_y,SO/W,SO9,SV,W,W-L%,WHIP,WP,cSho,tSho
2812,TEX,50,27.4,4.55,162,6163,5453,737,1308,266,...,1121,2.28,7.1,42,67,0.414,1.403,57,0.0,5.0
2813,TOR,63,28.9,4.38,162,6076,5477,709,1336,320,...,1298,2.36,8.1,39,73,0.451,1.414,71,0.0,3.0
2814,WSN,53,27.6,4.76,162,6288,5517,771,1402,284,...,1417,2.91,8.8,40,82,0.506,1.250,42,1.0,7.0


In [23]:
df.shape

(2815, 65)

In [24]:
df.Year.value_counts()

1884    33
2017    30
2015    30
1998    30
2002    30
2004    30
2006    30
2008    30
2010    30
2012    30
2014    30
2016    30
2018    30
2005    30
2013    30
1999    30
2001    30
2003    30
2007    30
2009    30
2011    30
1993    28
1997    28
1996    28
1994    28
1995    28
1992    26
1990    26
1988    26
1986    26
        ..
1941    16
1905    16
1911    16
1901    16
1883    16
1957    16
1955    16
1953    16
1951    16
1947    16
1949    16
1885    16
1887    16
1889    16
1882    14
1892    12
1897    12
1895    12
1893    12
1894    12
1896    12
1898    12
1899    12
1881     8
1900     8
1879     8
1880     8
1876     8
1877     6
1878     6
Name: Year, Length: 142, dtype: int64